In [12]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

store_list    = ["ricardoeletro", "magazineluiza"]

dataset_pages = f"data/heuristic_pages/*/*"
dataset_links = f"data/heusristic_links/heusristic_links/*"

agent = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          'Accept-Language': 'pt-BR'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:

url = r"https://www.amazon.com.br/Smartphone-Xiaomi-64GB-Piano-Black/dp/B07QH823FV/ref=pd_sbs_107_5/138-4765066-9703910?_encoding=UTF8&pd_rd_i=B07QH823FV&pd_rd_r=307fed63-71d8-4c9e-aacc-82a0ddc80cd9&pd_rd_w=x0jYi&pd_rd_wg=88HJ1&pf_rd_p=80c6065d-57d3-41bf-b15e-ee01dd80424f&pf_rd_r=7N8T5AJWRZ6VPNQTB88Z&psc=1&refRID=7N8T5AJWRZ6VPNQTB88Z"
code        = requests.get(url, headers=agent)
plain       = code.text
s           = BeautifulSoup(plain, "html.parser")

In [141]:
def levenshtein(word1, word2):
    """
    https://medium.com/@yash_agarwal2/soundex-and-levenshtein-distance-in-python-8b4b56542e9e
    https://en.wikipedia.org/wiki/Levenshtein_distance
    :param word1:
    :param word2:
    :return:
    """
    word2 = word2.lower()
    word1 = word1.lower()
    matrix = [[0 for x in range(len(word2) + 1)] for x in range(len(word1) + 1)]

    for x in range(len(word1) + 1):
        matrix[x][0] = x
    for y in range(len(word2) + 1):
        matrix[0][y] = y

    for x in range(1, len(word1) + 1):
        for y in range(1, len(word2) + 1):
            if word1[x - 1] == word2[y - 1]:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1],
                    matrix[x][y - 1] + 1
                )
            else:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1] + 1,
                    matrix[x][y - 1] + 1
                )

    return matrix[len(word1)][len(word2)]

def nomr_l(word1, word2):
    return levenshtein(word1, word2) / max(len(word1), len(word2))

def html_to_np(table):
    result = []
    all_rows = table.find_all('tr')
    for row in all_rows:
        result.append([])
        all_cols = row.find_all('td')
        for col in all_cols:
            col_text = [s for s in col.find_all(text=True)]
            col_text  = ''.join(col_text) # it should not change anything
            result[-1].append(col_text)
    return np.array(result)


def np_to_dict(table, fields_dict):
    for cols in table:
        if len(cols) != 2:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[0]) <= 0.2:
                fields_dict[key] = cols[1].replace("\n",'')
    return fields_dict


            
fields_amazom = {'Preço': '',
        'Sistema operacional': '',
        'RAM': '',
        'Capacidade de armazenamento digital': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho da tela':'',
        'Avaliações de clientes': ''
       }

In [142]:
tb = html_to_np(s)
np_to_dict(tb, fields_amazom)

{'Preço': 'R$1.998,00',
 'Sistema operacional': 'android',
 'RAM': '6',
 'Capacidade de armazenamento digital': '6 GB',
 'Modelo': 'MZB7606EU',
 'Cor': 'Preto',
 'Tamanho da tela': '5.97 polegadas',
 'Avaliações de clientes': '4.7 de 5 estrelas      109 avaliações de clientes    '}